##  Toronto Neighborhoods Segmenting and Clustering  

#### *Author: Mohammad Sayeb*

#### Let's import the relevant modules

In [1]:
import pandas as pd #pandas data frame from efficient dataframe manipulation
import numpy as np #for dealing with multidimensional arrays and matrices
import plotly #visualization tool
import matplotlib #visualization tool
import matplotlib.pyplot as plt # for visualization purposes
import requests #requesting information from webpages 
import bs4 #beautiful soup library for website scraping 
from bs4 import BeautifulSoup #scraping tool
import lxml #needed to convert html bs4 object to data frame

In [2]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find_all('table')

In [4]:
df = pd.read_html(str(table))[0]

In [21]:
df.shape

(180, 3)

### Ignore the rows that don't have an assigned borough

In [28]:
df = df[df['Borough']!='Not assigned']
print (df.shape)
df.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


###  Now we would like to put the neighborhoods that belong to the same postal code in the same row separated by commas. The fist step here is to see if there are any duplicated values for the Postal Codes.

In [31]:
df.duplicated(subset=None, keep='first')[df.duplicated(subset=None, keep='first')==True]

Series([], dtype: bool)

### We see that there are no duplicated postal code values we don't need to worry about combining the Neighbourhoods that belong to the same Postal Code into one row separated by commas

### If a cell has a borough but a Not assigned neighbourhood, then the neighborhood will be the same as the borough

In [34]:
df[df['Neighbourhood']=='Not assigned']

(0, 3)

### There are no rows with a Not assinged neighbourhood

In [36]:
print ('the data frame has {} rows and {} columns'.format(df.shape[0],df.shape[1]))
df.shape

the data frame has 103 rows and 3 columns


(103, 3)